In [3]:
import pandas as pd # 데이터 프레임을 쓰기 위해 사용함

with open("eng_raw.txt", "r", encoding="utf-8") as file:
    lines = file.read()
data = lines.split("$$") # txt 파일에 $$로 행 구분이 되어 있음
df = pd.DataFrame([line.strip().split("||") for line in data[1:-1]], columns=data[0].strip().split("||")) # 마지막 행은 "\n"이라서 추가 안함
df = df.fillna("0") # 결측치를 전부 0으로 대체함
df.head()

,제목,분야,본문
0,With more than 800 antisemitic acts since Oct....,정치,The Education Department s Office for Civil Ri...
1,Black Friday shopping deals have started. Here...,경제,In the last several years retailers have star...
2,Joe Biden falls behind Donald Trump with young...,선거_사회,WASHINGTON President Joe Biden is trailing f...
3,The pre-workout supplement market is exploding...,생활,Various pre workout brands contain various ing...
4,Has Elon Musk gone too far? Outrage grows over...,기술,It is unacceptable to repeat the hideous lie b...


# **<span style="color:blue">데이터 전처리</span>**
---

In [4]:
import re # 뉴스 기사의 특수문자를 제거하기 위해 사용함
from gensim.summarization.summarizer import summarize # 텍스트 요약 라이브러리
from googletrans import Translator # 번역을 할 수 있는 라이브러리

df["제목"] = df["제목"].apply(lambda x: x.strip().replace("\n"," ").replace("\t"," ")) # 제목 열 특수문자 처리
df["본문"] = df["본문"].apply(lambda x:re.sub(r'[^\w\s.]', ' ', x).replace('.', '. ').replace('\xa0', '. ').strip()) # 뉴스 본문 특수문자 처리

df["요약"] = df["본문"].apply(lambda x: summarize(x, word_count=50))

for i in range(len(df["요약"])): # 요약본이 안 생겼을 경우는 본문을 요약 칸에 채움
    if df["요약"][i] == "":
        df["요약"][i] = df["본문"][i]
        
df = df.drop(columns=['본문']) # 뉴스의 본문 변수는 필요없기 때문에 삭제함

df["번역_kor"] =" " # 번역 열 생성

def func(x): # 선정된 뉴스의 요약을 번역하는 함수 설정
    translator = Translator() # 번역기 객체 생성
    sentence = x.replace("\n", " ") # 번역을 하기 위해 요약 변수값을 조정
    kor = translator.translate(sentence,  dest = "ko") # 번역기 객체를 사용해 한국어 -> 영어 번역
    return kor.text # 번역된 텍스트를 반환

df["번역_kor"] = df["요약"].apply(func)

# 이미지 이름 열 저장 -> DB에 쉽게 저장하기 위함
df["이미지"] = " "
for i in range(6):
    name = f"eng_image_{i}"
    df["이미지"][i] = name

df.head()

,제목,분야,요약,번역_kor,이미지
0,With more than 800 antisemitic acts since Oct....,정치,The Education Department s Office for Civil Ri...,교육부의 민권 사무소는 또한 차별 불만 양식을 업데이트하여 대학과 대학과 연방 기금...,eng_image_0
1,Black Friday shopping deals have started. Here...,경제,In the last several years retailers have star...,지난 몇 년 동안 소매 업체는 Black Friday 또는 Cyber Monday ...,eng_image_1
2,Joe Biden falls behind Donald Trump with young...,선거_사회,WASHINGTON President Joe Biden is trailing f...,조 비덴 (Joe Biden) 워싱턴 대통령은 일요일에 발표 된 NBC 뉴스 여론 ...,eng_image_2
3,The pre-workout supplement market is exploding...,생활,Various pre workout brands contain various ing...,다양한 사전 운동 브랜드에는 피로를 막고 에너지 수준을 높이기 위해 함께 일하는 것...,eng_image_3
4,Has Elon Musk gone too far? Outrage grows over...,기술,It is unacceptable to repeat the hideous lie b...,미국 역사상 가장 치명적인 반유대주의 행위 뒤에 홀로 코스트 이후 유대인들에게 가장...,eng_image_4


# **<span style="color:blue">결과물을 CSV와 파일로 저장</span>**
---

In [5]:
import numpy as np

df.to_csv("eng_scraping.csv", index=False)

df_numpy = df.to_numpy()
file = open("eng_scraping.txt", "w")

for i in range(len(df_numpy)):
    file.write(f"{i}||")
    file.write("||".join(map(str,list(df_numpy[i]))))
    file.write("&&")

file.close()